# Laptop price prediction
Source: https://platform.stratascratch.com/data-projects/laptop-price-prediction

## (1) Imports and setups

In [13]:
import pandas as pd
import numpy as np
import json

In [8]:
train_dt = pd.read_json("./datasets/train_dataset.json")
val_dt = pd.read_json("./datasets/val_dataset.json")
test_dt = pd.read_json("./datasets/test_dataset.json")

In [43]:
print(train_dt.info())

<class 'pandas.core.frame.DataFrame'>
Index: 4711 entries, 7233 to 6037
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   graphic card type       4417 non-null   object 
 1   communications          4261 non-null   object 
 2   resolution (px)         4361 non-null   object 
 3   CPU cores               4711 non-null   object 
 4   RAM size                4457 non-null   object 
 5   operating system        4335 non-null   object 
 6   drive type              4454 non-null   object 
 7   input devices           4321 non-null   object 
 8   multimedia              4310 non-null   object 
 9   RAM type                4212 non-null   object 
 10  CPU clock speed (GHz)   4181 non-null   float64
 11  CPU model               4389 non-null   object 
 12  state                   4711 non-null   object 
 13  drive memory size (GB)  4439 non-null   float64
 14  warranty                4711 non-null   ob

In [44]:
print(val_dt.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1571 entries, 3849 to 4277
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   graphic card type       1471 non-null   object 
 1   communications          1409 non-null   object 
 2   resolution (px)         1442 non-null   object 
 3   CPU cores               1571 non-null   object 
 4   RAM size                1478 non-null   object 
 5   operating system        1437 non-null   object 
 6   drive type              1485 non-null   object 
 7   input devices           1426 non-null   object 
 8   multimedia              1420 non-null   object 
 9   RAM type                1394 non-null   object 
 10  CPU clock speed (GHz)   1375 non-null   float64
 11  CPU model               1467 non-null   object 
 12  state                   1571 non-null   object 
 13  drive memory size (GB)  1467 non-null   float64
 14  warranty                1571 non-null   ob

In [42]:
print(test_dt.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1571 entries, 5124 to 1371
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   graphic card type       1469 non-null   object 
 1   communications          1401 non-null   object 
 2   resolution (px)         1442 non-null   object 
 3   CPU cores               1571 non-null   object 
 4   RAM size                1468 non-null   object 
 5   operating system        1431 non-null   object 
 6   drive type              1479 non-null   object 
 7   input devices           1428 non-null   object 
 8   multimedia              1415 non-null   object 
 9   RAM type                1383 non-null   object 
 10  CPU clock speed (GHz)   1361 non-null   float64
 11  CPU model               1464 non-null   object 
 12  state                   1571 non-null   object 
 13  drive memory size (GB)  1466 non-null   float64
 14  warranty                1571 non-null   ob

## (2) Data manipulation

In [ ]:
# TO DO - Explode and pivot 'communication'

## (3) EDA

In [19]:
numvars = train_dt.select_dtypes(include=[np.number]).columns.to_list()
catvars = train_dt.select_dtypes(include=[object]).columns.to_list()
print("Numerical variables: ", numvars)
print("Categorical variables: ", catvars)

Numerical variables:  ['CPU clock speed (GHz)', 'drive memory size (GB)', 'buynow_price']
Categorical variables:  ['graphic card type', 'communications', 'resolution (px)', 'CPU cores', 'RAM size', 'operating system', 'drive type', 'input devices', 'multimedia', 'RAM type', 'CPU model', 'state', 'warranty', 'screen size']


Check for missing values

In [ ]:
pd.concat(
    [train_dt.isna().sum(), train_dt.isna().mean()],
    axis=1,
    keys=["Count", "Proportion"],
)

,Count,Proportion
graphic card type,294,0.062407
communications,450,0.095521
resolution (px),350,0.074294
CPU cores,0,0.000000
RAM size,254,0.053916
operating system,376,0.079813
drive type,257,0.054553
input devices,390,0.082785
multimedia,401,0.085120
RAM type,499,0.105922


Univariate Analysis

In [ ]:
train_dt[numvars].describe(percentiles=[0.95])

,CPU clock speed (GHz),drive memory size (GB),buynow_price
count,4181.000000,4439.000000,4711.000000
mean,2.342057,652.619284,3495.831195
std,0.386298,467.657354,1727.933306
min,0.800000,0.000000,429.000000
50%,2.500000,500.000000,3184.000000
95%,2.800000,1500.000000,6779.000000
max,3.900000,2960.000000,15472.650000


In [40]:
train_dt["communications"].value_counts()

communications
[wi-fi, bluetooth, lan 10/100/1000 mbps]                                                                                              1128
[bluetooth, lan 10/100 mbps]                                                                                                           656
[bluetooth, lan 10/100/1000 mbps, lan 10/100 mbps, intel wireless display (widi), nfc (near field communication), modem 3g (wwan)]     429
[bluetooth, lan 10/100/1000 mbps]                                                                                                      395
[wi-fi, bluetooth, lan 10/100 mbps]                                                                                                    306
                                                                                                                                      ... 
[wi-fi 802.11 a/b/g/n/ac, wi-fi 802.11 b/g/n, wi-fi 802.11 a/b/g/n, bluetooth, lan 10/100 mbps]                                          1
[lan 10/100/

In [39]:
for i in catvars:
    print(i)
    print(train_dt[i].unique())
    # print(f"{i}: N={train_dt[i].nunique()} {train_dt[i].unique()}")

graphic card type
['dedicated graphics' None 'integrated graphics']
communications


TypeError: unhashable type: 'list'